TP4.1
Ejercicio1:

In [ ]:
import requests

def obtener_datos_eventos():
    url = "https://www.cultura.gob.ar/api/v2.0/organismos/"
    try:
        response = requests.get(url)
        response.raise_for_status()  # Verificar si la solicitud fue exitosa
        data = response.json()
        return data
    except requests.exceptions.RequestException as e:
        print("Error al obtener los datos:", e)
        return None

def eventos_dependen_ministerio(data):
    eventos_ministerio = 0
    for evento in data:
        if evento.get("depende_de") == "Ministerio de Cultura de la Nación":
            eventos_ministerio += 1
    return eventos_ministerio

def eventos_sin_dependencia(data):
    eventos_sin_dependencia = []
    for evento in data:
        if not evento.get("depende_de"):
            eventos_sin_dependencia.append(evento.get("nombre"))
    return eventos_sin_dependencia

def listar_nombres_eventos(data):
    nombres_eventos = []
    for evento in data:
        nombres_eventos.append(evento.get("nombre"))
    return nombres_eventos

def crear_diccionario_id_nombre(data):
    diccionario = {}
    for evento in data:
        diccionario[evento.get("id")] = evento.get("nombre")
    return diccionario

if __name__ == "__main__":
    datos_eventos = obtener_datos_eventos()

    if datos_eventos:
        # Tarea 1: Indicar cuántos eventos dependen del "Ministerio de Cultura de la Nación"
        eventos_ministerio = eventos_dependen_ministerio(datos_eventos)
        print("Eventos dependen del Ministerio de Cultura de la Nación:", eventos_ministerio)

        # Tarea 2: Mostrar el nombre del o los eventos donde no figura de quién depende.
        eventos_sin_dependencia = eventos_sin_dependencia(datos_eventos)
        if eventos_sin_dependencia:
            print("Eventos sin dependencia:")
            for evento in eventos_sin_dependencia:
                print("-", evento)

        # Tarea 3: Listar el nombre de todos los eventos descritos
        nombres_eventos = listar_nombres_eventos(datos_eventos)
        print("Nombres de todos los eventos descritos:")
        for nombre_evento in nombres_eventos:
            print("-", nombre_evento)

        # Tarea 4: Crear un diccionario usando como clave:valor a id:nombreEvento
        diccionario_id_nombre = crear_diccionario_id_nombre(datos_eventos)
        print("Diccionario id:nombreEvento:")
        for id_evento, nombre_evento in diccionario_id_nombre.items():
            print(f"{id_evento}: {nombre_evento}")
    else:
        print("No se pudieron obtener los datos de los eventos.")


Ejercicio2:

In [ ]:
import requests
import json

def obtener_datos_museos():
    url = "https://www.cultura.gob.ar/api/v2.0/museos/"
    try:
        response = requests.get(url)
        response.raise_for_status()  # Verificar si la solicitud fue exitosa
        data = response.json()
        return data.get("results")
    except requests.exceptions.RequestException as e:
        print("Error al obtener los datos:", e)
        return None

def agregar_provincia_a_museos(museos):
    # Seleccionamos algunas provincias arbitrariamente para asignar a los museos
    provincias = ["Buenos Aires", "Córdoba", "Santa Fe", "Mendoza", "Tucumán", "Salta", "Chubut", "Entre Ríos", "Jujuy"]
    for i, museo in enumerate(museos):
        provincia = provincias[i % len(provincias)]
        museo["provincia"] = provincia
    return museos

def generar_nuevo_archivo_con_provincias(museos):
    nuevos_museos = agregar_provincia_a_museos(museos)
    with open("museos_con_provincias.json", "w") as file:
        json.dump(nuevos_museos, file, indent=2)

def generar_nuevo_archivo_solo_datos(museos):
    nuevos_museos = [{ "id": museo.get("id"), 
                       "nombre": museo.get("nombre"), 
                       "direccion": museo.get("direccion"), 
                       "telefono": museo.get("telefono") 
                     } for museo in museos]
    with open("museos_solo_datos.json", "w") as file:
        json.dump(nuevos_museos, file, indent=2)

if __name__ == "__main__":
    datos_museos = obtener_datos_museos()

    if datos_museos:
        # Tarea 1: Generar un nuevo archivo JSON con la provincia en donde se encuentra cada museo.
        generar_nuevo_archivo_con_provincias(datos_museos)
        print("Se ha generado el archivo 'museos_con_provincias.json'.")

        # Tarea 2: Generar un nuevo archivo JSON donde los únicos datos del campo results sean id, nombre, dirección y teléfono por cada museo.
        generar_nuevo_archivo_solo_datos(datos_museos)
        print("Se ha generado el archivo 'museos_solo_datos.json'.")
    else:
        print("No se pudieron obtener los datos de los museos.")


In [ ]:
import requests
import json

# Funciones para obtener datos de las APIs

def obtener_provincias():
    url = "https://apis.datos.gob.ar/georef/api/provincias?campos=id,nombre"
    response = requests.get(url)
    data = response.json()
    return data["provincias"]

def obtener_localidades(provincia_id):
    url = f"https://apis.datos.gob.ar/georef/api/municipios?provincia={provincia_id}&campos=id,nombre&max=100"
    response = requests.get(url)
    data = response.json()
    return data["municipios"]

# Funciones para el registro y consulta de eventos

def registro_evento():
    evento = {}
    evento["nombre_evento"] = input("Ingrese el nombre del evento: ")
    evento["descripcion"] = input("Ingrese una descripción del evento: ")
    evento["dia"] = int(input("Ingrese el día del evento (1-31): "))
    evento["mes"] = input("Ingrese el mes del evento: ")
    provincias = obtener_provincias()
    print("Provincias disponibles:")
    for provincia in provincias:
        print(f"{provincia['id']}: {provincia['nombre']}")
    provincia_id = int(input("Ingrese el ID de la provincia donde se realizará el evento: "))
    localidades = obtener_localidades(provincia_id)
    print("Localidades disponibles:")
    for localidad in localidades:
        print(f"{localidad['id']}: {localidad['nombre']}")
    evento["provincia"] = provincias[provincia_id-1]["nombre"]
    localidad_id = int(input("Ingrese el ID de la localidad donde se realizará el evento: "))
    evento["localidad"] = localidades[localidad_id-1]["nombre"]
    
    with open("eventos.json", "a") as file:
        json.dump(evento, file)
        file.write("\n")
    print("Evento registrado exitosamente.")

def consulta_eventos():
    with open("eventos.json", "r") as file:
        eventos = [json.loads(line.strip()) for line in file]

    print("1. Eventos en mi localidad")
    print("2. Todos los eventos")
    print("3. Eventos entre dos fechas")
    opcion = int(input("Seleccione una opción: "))

    if opcion == 1:
        provincia = input("Ingrese el nombre de su provincia: ")
        localidad = input("Ingrese el nombre de su localidad: ")
        eventos_localidad = [evento for evento in eventos if evento["provincia"] == provincia and evento["localidad"] == localidad]
        if eventos_localidad:
            print("Eventos en su localidad:")
            for evento in eventos_localidad:
                print(evento)
        else:
            print("No se encontraron eventos en su localidad.")
    elif opcion == 2:
        print("Todos los eventos:")
        for evento in eventos:
            print(evento)
    elif opcion == 3:
        dia_inicio = int(input("Ingrese el día de inicio del intervalo (1-31): "))
        mes_inicio = input("Ingrese el mes de inicio del intervalo: ")
        dia_fin = int(input("Ingrese el día de fin del intervalo (1-31): "))
        mes_fin = input("Ingrese el mes de fin del intervalo: ")
        eventos_intervalo = [evento for evento in eventos if evento["mes"] == mes_inicio and evento["dia"] >= dia_inicio
                             or evento["mes"] == mes_fin and evento["dia"] <= dia_fin
                             or mes_inicio < evento["mes"] < mes_fin]
        if eventos_intervalo:
            print("Eventos en el intervalo seleccionado:")
            for evento in eventos_intervalo:
                print(evento)
        else:
            print("No se encontraron eventos en el intervalo seleccionado.")
    else:
        print("Opción inválida.")

# Menú principal del programa

if __name__ == "__main__":
    print("¿A dónde voy en las vacaciones de invierno? Gratis…")
    while True:
        print("\nMenú:")
        print("1. Registro de evento")
        print("2. Consulta de eventos")
        print("3. Salir")
        opcion = int(input("Seleccione una opción: "))

        if opcion == 1:
            registro_evento()
        elif opcion == 2:
            consulta_eventos()
        elif opcion == 3:
            break
        else:
            print("Opción inválida.")
